In [1]:
# Imports
from gensim import downloader as api
import csv
import random


In [2]:
# Task 1

# Load pre-trained model
model = api.load("word2vec-google-news-300") 

In [3]:
# Global Variables
correct = 0 # variable to count correct guess words
valid = 0 # variable to count valid guess words
label = "guess" # variable to label system's guess result
score_dict = {} # dictionary for score-word pairs


# Open synonym.csv in read mode
with open('synonym.csv', mode='r') as dataset:

    csv_reader = csv.reader(dataset, delimiter=',')
    linecount = 0 # variable to count number of lines
    
    # Open word2vec-google-news-300-details.csv in write mode
    with open('word2vec-google-news-300-details.csv', mode='w', newline='') as details:
        csv_writer = csv.writer(details)

        # For every row in synonym.csv
        for row in csv_reader:
            # csv Headings
            if linecount == 0:
                csv_writer.writerow(['question-word', 'answer-word', 'guess-word', 'label'])
            else:
                
                score_dict = {} # dictionary for score-word pairs
                inModel = True # variable used for checking valid guesses
                random_word = random.randint(2,5) # random integer between 2 and 5 (inclusive)

                # If either question-word or all four guess-words are not in model, guess randomly
                if(row[0] not in model or (row[2] not in model and row[3] not in model and row[4] not in model and row[5] not in model)):
                    inModel = False
                    guess_word = row[random_word]

                else: 
                    inModel = True

                    # Compute similarities of scores of question-word and guess-word and append them as key-value pairs to the dictionary
                    score1 = model.similarity(row[0],row[2])
                    score_dict[score1] = row[2]
                    
                    score2 = model.similarity(row[0],row[3])
                    score_dict[score2] = row[3]
                    
                    score3 = model.similarity(row[0],row[4])
                    score_dict[score3] = row[4]
                    
                    score4 = model.similarity(row[0],row[5])
                    score_dict[score4] = row[5]

                    guess_key = sorted(score_dict.keys())[-1] # sort the final dictionary and get the highest key (score)
                    guess_word = score_dict[guess_key] # get the word with the highest score

                # Label the system's guess
                if(inModel == False):
                    label = "guess"      
                elif(guess_word == row[1]):
                    correct += 1
                    valid += 1
                    label = "correct"
                else:
                    valid += 1
                    label = "wrong"

                csv_writer.writerow([row[0], row[1], guess_word, label]) # write final result to csv
            
            score_dict.clear() # clear the dictionary to use in the next row
            linecount +=1 # increase the line count

In [4]:
# Open analysis.csv in write mode
with open('analysis.csv', mode='w', newline='') as details:
        csv_writer = csv.writer(details)
        model_name = "word2vec-google-news-300"
        vocab_size = 3000000
        accuracy = correct/valid
        csv_writer.writerow(["Model Name", "Vocabulary Size", "Number of Correct Labels","Number of Valid Answers", "Model Accuracy \n"]) # csv Headings
        csv_writer.writerow([model_name, vocab_size, correct, valid, accuracy]) # analysis results
        csv_writer.writerow("")

In [5]:
# Task 2

# Load all pre-trained models
C1E1 = api.load("glove-wiki-gigaword-300")
#C2E1 = api.load("")
C3E2 = api.load("glove-twitter-200")
C3E3 = api.load("glove-twitter-25")

In [6]:
# Task 2: C1E1

# Global Variables
correct = 0 # variable to count correct guess words
valid = 0 # variable to count valid guess words
label = "guess" # variable to label system's guess result
score_dict = {} # dictionary for score-word pairs


# Open synonym.csv in read mode
with open('synonym.csv', mode='r') as dataset:

    csv_reader = csv.reader(dataset, delimiter=',')
    linecount = 0 # variable to count number of lines
    
    # Open word2vec-google-news-300-details.csv in write mode
    with open('glove-wiki-gigaword-300-details.csv', mode='w', newline='') as details:
        csv_writer = csv.writer(details)

        # For every row in synonym.csv
        for row in csv_reader:
            # csv Headings
            if linecount == 0:
                csv_writer.writerow(['question-word', 'answer-word', 'guess-word', 'label'])
            else:
                
                score_dict = {} # dictionary for score-word pairs
                inModel = True # variable used for checking valid guesses
                random_word = random.randint(2,5) # random integer between 2 and 5 (inclusive)

                # If either question-word or all four guess-words are not in C1E1, guess randomly
                if(row[0] not in C1E1 or (row[2] not in C1E1 or row[3] not in C1E1 or row[4] not in C1E1 or row[5] not in C1E1)):
                    inModel = False
                    guess_word = row[random_word]

                else: 
                    inModel = True

                    # Compute similarities of scores of question-word and guess-word and append them as key-value pairs to the dictionary
                    score1 = C1E1.similarity(row[0],row[2])
                    score_dict[score1] = row[2]
                    
                    score2 = C1E1.similarity(row[0],row[3])
                    score_dict[score2] = row[3]
                    
                    score3 = C1E1.similarity(row[0],row[4])
                    score_dict[score3] = row[4]
                    
                    score4 = C1E1.similarity(row[0],row[5])
                    score_dict[score4] = row[5]

                    guess_key = sorted(score_dict.keys())[-1] # sort the final dictionary and get the highest key (score)
                    guess_word = score_dict[guess_key] # get the word with the highest score

                # Label the system's guess
                if(inModel == False):
                    label = "guess"      
                elif(guess_word == row[1]):
                    correct += 1
                    valid += 1
                    label = "correct"
                else:
                    valid += 1
                    label = "wrong"

                csv_writer.writerow([row[0], row[1], guess_word, label]) # write final result to csv
            
            score_dict.clear() # clear the dictionary to use in the next row
            linecount +=1 # increase the line count


# Open analysis.csv in append mode
with open('analysis.csv', mode='a', newline='') as details:
        csv_writer = csv.writer(details)
        model_name = "glove-wiki-gigaword-300"
        vocab_size = 400000
        accuracy = correct/valid
        csv_writer.writerow([model_name, vocab_size, correct, valid, accuracy]) # analysis results
        csv_writer.writerow("")

In [7]:
# Task 2: C2E1


In [8]:
# Task 2: C3E2

# Global Variables
correct = 0 # variable to count correct guess words
valid = 0 # variable to count valid guess words
label = "guess" # variable to label system's guess result
score_dict = {} # dictionary for score-word pairs


# Open synonym.csv in read mode
with open('synonym.csv', mode='r') as dataset:

    csv_reader = csv.reader(dataset, delimiter=',')
    linecount = 0 # variable to count number of lines
    
    # Open word2vec-google-news-300-details.csv in write mode
    with open('glove-twitter-200-details.csv', mode='w', newline='') as details:
        csv_writer = csv.writer(details)

        # For every row in synonym.csv
        for row in csv_reader:
            # csv Headings
            if linecount == 0:
                csv_writer.writerow(['question-word', 'answer-word', 'guess-word', 'label'])
            else:
                
                score_dict = {} # dictionary for score-word pairs
                inModel = True # variable used for checking valid guesses
                random_word = random.randint(2,5) # random integer between 2 and 5 (inclusive)

                # If either question-word or all four guess-words are not in C1E1, guess randomly
                if(row[0] not in C3E2 or (row[2] not in C3E2 or row[3] not in C3E2 or row[4] not in C3E2 or row[5] not in C3E2)):
                    inModel = False
                    guess_word = row[random_word]

                else: 
                    inModel = True

                    # Compute similarities of scores of question-word and guess-word and append them as key-value pairs to the dictionary
                    score1 = C3E2.similarity(row[0],row[2])
                    score_dict[score1] = row[2]
                    
                    score2 = C3E2.similarity(row[0],row[3])
                    score_dict[score2] = row[3]
                    
                    score3 = C3E2.similarity(row[0],row[4])
                    score_dict[score3] = row[4]
                    
                    score4 = C3E2.similarity(row[0],row[5])
                    score_dict[score4] = row[5]

                    guess_key = sorted(score_dict.keys())[-1] # sort the final dictionary and get the highest key (score)
                    guess_word = score_dict[guess_key] # get the word with the highest score

                # Label the system's guess
                if(inModel == False):
                    label = "guess"      
                elif(guess_word == row[1]):
                    correct += 1
                    valid += 1
                    label = "correct"
                else:
                    valid += 1
                    label = "wrong"

                csv_writer.writerow([row[0], row[1], guess_word, label]) # write final result to csv
            
            score_dict.clear() # clear the dictionary to use in the next row
            linecount +=1 # increase the line count


# Open analysis.csv in append mode
with open('analysis.csv', mode='a', newline='') as details:
        csv_writer = csv.writer(details)
        model_name = "glove-twitter-200"
        vocab_size = 1193514
        accuracy = correct/valid
        csv_writer.writerow([model_name, vocab_size, correct, valid, accuracy]) # analysis results
        csv_writer.writerow("")

In [9]:
# Task 2: C3E3

# Global Variables
correct = 0 # variable to count correct guess words
valid = 0 # variable to count valid guess words
label = "guess" # variable to label system's guess result
score_dict = {} # dictionary for score-word pairs


# Open synonym.csv in read mode
with open('synonym.csv', mode='r') as dataset:

    csv_reader = csv.reader(dataset, delimiter=',')
    linecount = 0 # variable to count number of lines
    
    # Open word2vec-google-news-300-details.csv in write mode
    with open('glove-twitter-25-details.csv', mode='w', newline='') as details:
        csv_writer = csv.writer(details)

        # For every row in synonym.csv
        for row in csv_reader:
            # csv Headings
            if linecount == 0:
                csv_writer.writerow(['question-word', 'answer-word', 'guess-word', 'label'])
            else:
                
                score_dict = {} # dictionary for score-word pairs
                inModel = True # variable used for checking valid guesses
                random_word = random.randint(2,5) # random integer between 2 and 5 (inclusive)

                # If either question-word or all four guess-words are not in C1E1, guess randomly
                if(row[0] not in C3E3 or (row[2] not in C3E3 or row[3] not in C3E3 or row[4] not in C3E3 or row[5] not in C3E3)):
                    inModel = False
                    guess_word = row[random_word]

                else: 
                    inModel = True

                    # Compute similarities of scores of question-word and guess-word and append them as key-value pairs to the dictionary
                    score1 = C3E3.similarity(row[0],row[2])
                    score_dict[score1] = row[2]
                    
                    score2 = C3E3.similarity(row[0],row[3])
                    score_dict[score2] = row[3]
                    
                    score3 = C3E3.similarity(row[0],row[4])
                    score_dict[score3] = row[4]
                    
                    score4 = C3E3.similarity(row[0],row[5])
                    score_dict[score4] = row[5]

                    guess_key = sorted(score_dict.keys())[-1] # sort the final dictionary and get the highest key (score)
                    guess_word = score_dict[guess_key] # get the word with the highest score

                # Label the system's guess
                if(inModel == False):
                    label = "guess"      
                elif(guess_word == row[1]):
                    correct += 1
                    valid += 1
                    label = "correct"
                else:
                    valid += 1
                    label = "wrong"

                csv_writer.writerow([row[0], row[1], guess_word, label]) # write final result to csv
            
            score_dict.clear() # clear the dictionary to use in the next row
            linecount +=1 # increase the line count


# Open analysis.csv in append mode
with open('analysis.csv', mode='a', newline='') as details:
        csv_writer = csv.writer(details)
        model_name = "glove-twitter-25"
        vocab_size = 1193514
        accuracy = correct/valid
        csv_writer.writerow([model_name, vocab_size, correct, valid, accuracy]) # analysis results
        csv_writer.writerow("")